In [82]:
# Clone the GitHub repository
!git clone https://github.com/geopan2000/Exploring-Mental-Health-Data.git

# Change directory to the cloned repository
%cd Exploring-Mental-Health-Data

Cloning into 'Exploring-Mental-Health-Data'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 48 (delta 14), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (48/48), 5.50 MiB | 3.05 MiB/s, done.
Resolving deltas: 100% (14/14), done.
/content/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data


In [83]:
import pandas as pd
!pip install xgboost
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
!pip install scikit-optimize
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import xgboost as xgb
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

!pip install catboost
from catboost import CatBoostClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.model_selection import KFold
import pandas as pd

In [84]:
train_set = pd.read_csv('/content/Exploring-Mental-Health-Data/data/train.csv')
test_set = pd.read_csv('/content/Exploring-Mental-Health-Data/data/test.csv')

In [85]:
train_set.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


# Data Inspection

In [86]:
#train_print = train_set.copy()
#train_print = train_print.drop(columns=['Depression'])

#for column in train_print.columns:
    #train_nan_ratio = train_print[column].isna().sum() / len(train_print)  # Proportion of NaN in train set
    #test_nan_ratio = test_set[column].isna().sum() / len(test_set)  # Proportion of NaN in test set
    #print(f"{column}  Train Set: {train_nan_ratio:.2%} Test Set: {test_nan_ratio:.2%}")

In [87]:
# Get the value counts for train and test sets
#train_counts = train_set['Profession'].value_counts()
#test_counts = test_set['Profession'].value_counts()

# Get a combined set of unique values from both train and test sets
#all_professions = set(train_counts.index).union(set(test_counts.index))

# Print the counts for each profession
#for profession in all_professions:
 #   train_count = train_counts.get(profession, 0)  # Get count from train, default to 0
  #  test_count = test_counts.get(profession, 0)   # Get count from test, default to 0
   # print(f"{profession}: Train_set={train_count} Test_set={test_count}")

# Feature Engineering:


In [88]:
train_id = train_set['id']

target_variable = train_set['Depression']
train_set.drop(columns=['Name','Depression'], inplace=True)
test_id = test_set['id']

test_set.drop(columns=['Name'], inplace=True)

train_set['Gender'] = train_set['Gender'].map({'Male' : 1, 'Female' : 0}) # Always trasnforming 2 option object types into binary
test_set['Gender'] = test_set['Gender'].map({'Male' : 1, 'Female' : 0})

train_set['Age'] = train_set['Age'] / 100 # Normalized by the Max value
train_set['Age'] = train_set['Age'].fillna(train_set['Age'].mean())
test_set['Age'] = test_set['Age'] / 100
test_set['Age'] = test_set['Age'].fillna(test_set['Age'].mean())

train_set['Working Professional or Student'] = train_set['Working Professional or Student'].map({'Working Professional' : 1, 'Student' : 0}) # Always trasnforming 2 option object types into binary
test_set['Working Professional or Student'] = test_set['Working Professional or Student'].map({'Working Professional' : 1, 'Student' : 0})

train_set['CGPA'] = train_set['CGPA'] /10
train_set['CGPA'] = train_set['CGPA'].fillna(0)
test_set['CGPA'] = test_set['CGPA'] /10
test_set['CGPA'] = test_set['CGPA'].fillna(0)

train_set['Have you ever had suicidal thoughts ?'] = train_set['Have you ever had suicidal thoughts ?'].map({'Yes' : 1, 'No' : 0}) # Always trasnforming 2 option object types into binary
test_set['Have you ever had suicidal thoughts ?'] = test_set['Have you ever had suicidal thoughts ?'].map({'Yes' : 1, 'No' : 0})

train_set['Work/Study Hours'] = train_set['Work/Study Hours'] / 24 # Normalized by the Max value
test_set['Work/Study Hours'] = test_set['Work/Study Hours'] / 24

train_set['Financial Stress'] = train_set['Financial Stress'] / 5 # Normalized by the Max value
train_set['Financial Stress'] = train_set['Financial Stress'].fillna(0)
test_set['Financial Stress'] = test_set['Financial Stress'] / 5
test_set['Financial Stress'] = test_set['Financial Stress'].fillna(0)

train_set['Family History of Mental Illness'] = train_set['Family History of Mental Illness'].map({'Yes' : 1, 'No' : 0}) # Always trasnforming 2 option object types into binary
test_set['Family History of Mental Illness'] = test_set['Family History of Mental Illness'].map({'Yes' : 1, 'No' : 0})

In [89]:
# Create the new feature 'Academic/Work Pressure'
train_set['Academic/Work Pressure'] = train_set.apply(
    lambda row: row['Academic Pressure'] if pd.notna(row['Academic Pressure']) and pd.isna(row['Work Pressure']) else
                row['Work Pressure'] if pd.isna(row['Academic Pressure']) and pd.notna(row['Work Pressure']) else
                (row['Academic Pressure'] + row['Work Pressure']) / 2 if pd.notna(row['Academic Pressure']) and pd.notna(row['Work Pressure']) else
                None,  # Placeholder for rows where both are NaN
    axis=1)

# Fill any remaining NaN in 'Academic/Work Pressure' with its mean
train_set['Academic/Work Pressure'] = train_set['Academic/Work Pressure'].fillna(train_set['Academic/Work Pressure'].mean())

train_set['Academic Pressure'] = train_set['Academic Pressure'] / 5
train_set['Academic Pressure'] = train_set['Academic Pressure'].fillna(0)

train_set['Work Pressure'] = train_set['Work Pressure'] / 5
train_set['Work Pressure'] = train_set['Work Pressure'].fillna(0)

In [90]:
# Create the new feature 'Academic/Work Pressure'
test_set['Academic/Work Pressure'] = test_set.apply(
    lambda row: row['Academic Pressure'] if pd.notna(row['Academic Pressure']) and pd.isna(row['Work Pressure']) else
                row['Work Pressure'] if pd.isna(row['Academic Pressure']) and pd.notna(row['Work Pressure']) else
                (row['Academic Pressure'] + row['Work Pressure']) / 2 if pd.notna(row['Academic Pressure']) and pd.notna(row['Work Pressure']) else
                None,  # Placeholder for rows where both are NaN
    axis=1)

# Fill any remaining NaN in 'Academic/Work Pressure' with its mean
test_set['Academic/Work Pressure'] = test_set['Academic/Work Pressure'].fillna(test_set['Academic/Work Pressure'].mean())

test_set['Academic Pressure'] = test_set['Academic Pressure'] / 5
test_set['Academic Pressure'] = test_set['Academic Pressure'].fillna(0)

test_set['Work Pressure'] = test_set['Work Pressure'] / 5
test_set['Work Pressure'] = test_set['Work Pressure'].fillna(0)

In [91]:
# Create the new feature Study/Job Satisfaction
train_set['Study/Job Satisfaction'] = train_set.apply(
    lambda row: row['Study Satisfaction'] if pd.notna(row['Study Satisfaction']) and pd.isna(row['Job Satisfaction']) else
                row['Job Satisfaction'] if pd.isna(row['Study Satisfaction']) and pd.notna(row['Job Satisfaction']) else
                (row['Study Satisfaction'] + row['Job Satisfaction']) / 2 if pd.notna(row['Study Satisfaction']) and pd.notna(row['Job Satisfaction']) else
                None,  # Placeholder for rows where both are NaN
    axis=1)

# Fill any remaining NaN in Study/Job Satisfaction with its mean
train_set['Study/Job Satisfaction'] = train_set['Study/Job Satisfaction'].fillna(train_set['Study/Job Satisfaction'].mean())

train_set['Study Satisfaction'] = train_set['Study Satisfaction'] / 5
train_set['Study Satisfaction'] = train_set['Study Satisfaction'].fillna(0)

train_set['Job Satisfaction'] = train_set['Job Satisfaction'] / 5
train_set['Job Satisfaction'] = train_set['Job Satisfaction'].fillna(0)

In [92]:
# Create the new feature Study/Job Satisfaction
test_set['Study/Job Satisfaction'] = test_set.apply(
    lambda row: row['Study Satisfaction'] if pd.notna(row['Study Satisfaction']) and pd.isna(row['Job Satisfaction']) else
                row['Job Satisfaction'] if pd.isna(row['Study Satisfaction']) and pd.notna(row['Job Satisfaction']) else
                (row['Study Satisfaction'] + row['Job Satisfaction']) / 2 if pd.notna(row['Study Satisfaction']) and pd.notna(row['Job Satisfaction']) else
                None,  # Placeholder for rows where both are NaN
    axis=1)

# Fill any remaining NaN in Study/Job Satisfaction with its mean
test_set['Study/Job Satisfaction'] = test_set['Study/Job Satisfaction'].fillna(test_set['Study/Job Satisfaction'].mean())

test_set['Study Satisfaction'] = test_set['Study Satisfaction'] / 5
test_set['Study Satisfaction'] = test_set['Study Satisfaction'].fillna(0)

test_set['Job Satisfaction'] = test_set['Job Satisfaction'] / 5
test_set['Job Satisfaction'] = test_set['Job Satisfaction'].fillna(0)

In [93]:
# Combine train and test value counts
train_counts = train_set['City'].value_counts()
test_counts = test_set['City'].value_counts()

# Create a combined count of professions
combined_counts = train_counts.add(test_counts, fill_value=0)

# Identify professions appearing less than 5 times in either set
low_frequency_values = combined_counts[combined_counts < 10].index.tolist()

# Map low-frequency values to 'None' in both train and test sets
train_set['City'] = train_set['City'].apply(lambda x: 'Other' if x in low_frequency_values else x)
test_set['City'] = test_set['City'].apply(lambda x: 'Other' if x in low_frequency_values else x)

In [94]:
# Combine train and test value counts
train_counts = train_set['Profession'].value_counts()
test_counts = test_set['Profession'].value_counts()

# Create a combined count of professions
combined_counts = train_counts.add(test_counts, fill_value=0)

# Identify professions appearing less than 5 times in either set
low_frequency_values = combined_counts[combined_counts < 10].index.tolist()

# Map low-frequency values to 'None' in both train and test sets
train_set['Profession'] = train_set['Profession'].apply(lambda x: 'Other' if x in low_frequency_values else x)
test_set['Profession'] = test_set['Profession'].apply(lambda x: 'Other' if x in low_frequency_values else x)

In [95]:
# Combine train and test value counts
train_counts = train_set['Degree'].value_counts()
test_counts = test_set['Degree'].value_counts()

# Create a combined count of professions
combined_counts = train_counts.add(test_counts, fill_value=0)

# Identify professions appearing less than 5 times in either set
low_frequency_values = combined_counts[combined_counts < 10].index.tolist()

# Map low-frequency values to 'None' in both train and test sets
train_set['Degree'] = train_set['Degree'].apply(lambda x: 'Other' if x in low_frequency_values else x)
test_set['Degree'] = test_set['Degree'].apply(lambda x: 'Other' if x in low_frequency_values else x)

In [96]:
train_set['City'] = train_set['City'].fillna('Other')
test_set['City'] = test_set['City'].fillna('Other')

train_set['Degree'] = train_set['Degree'].fillna('Other')
test_set['Degree'] = test_set['Degree'].fillna('Other')

train_set['Profession'] = train_set['Profession'].fillna('Other')
test_set['Profession'] = test_set['Profession'].fillna('Other')

In [97]:
 #Define a function to encode Sleep Duration
def encode_sleep_duration(value):
    # Check for NaN or None
    if pd.isna(value):
        return -1  # Placeholder for missing or invalid values

    # Ensure the value is a string
    value = str(value).strip()

    # Encoding logic
    if 'Less than 5 hours' in value or value in ['3-4 hours', '4-5 hours', '2-3 hours']:
        return 1  # < 5 hours
    elif '5-6 hours' in value:
        return 2  # 5-6 hours
    elif '6-7 hours' in value or '6-8 hours' in value:
        return 3  # 6-7 hours
    elif '7-8 hours' in value:
        return 4  # 7-8 hours
    elif 'More than 8 hours' in value or '8-9 hours' in value or '9-11 hours' in value:
        return 5  # > 8 hours
    else:
        return -1  # Placeholder for other invalid or unclear values

sleep_mapping = {
    'More than 8 hours': '>8',
    'Less than 5 hours': '<5',
    'Moderate': '5-6 hours',
    '9-6 hours': '6-9 hours',

    '10-6 hours': '6-10 hours',
    'than 5 hours': '4-6 hours',
    'Unhealthy': '4-6 hours',
    '9-5 hours': '5-9 hours',
    '9-5': '5-9 hours',
    '8-89 hours': '8-9 hours',

    # Ambiguous or irrelevant entries
    'Sleep_Duration': None, '40-45 hours': None, '55-66 hours': None, 'Indore': None,
    '45': None, '35-36 hours': None, 'No': None, 'Indore': None, '49 hours': None,
    'Work_Study_Hours': None, '45-48 hours': None, 'Pune': None, 'Soham': None,
    '0': None, 'Meerut': None, '60-65 hours': None, 'Vivan': None,
    'Have_you_ever_had_suicidal_thoughts': None, '20-21 hours': None, '50-75 hours': None
}

# Define range extraction function
def extract_range(value):
    try:
        if '-' in value:
            parts = value.split('-')
            return int(parts[0]), int(parts[1].split()[0])
        elif '<' in value:
            return 3, 5
        elif '>' in value:
            lower_bound = int(value.split('>')[1].split()[0])
            return lower_bound, 10
        elif value.isdigit():
            num = int(value)
            return num, num
        else:
            return None, None
    except:
        return None, None

In [98]:
train_set['Sleep Duration'] = train_set['Sleep Duration'].map(sleep_mapping)
test_set['Sleep Duration'] = test_set['Sleep Duration'].map(sleep_mapping)

train_set['Sleep_Duration_Encoded'] = train_set['Sleep Duration'].apply(encode_sleep_duration)
test_set['Sleep_Duration_Encoded'] = test_set['Sleep Duration'].apply(encode_sleep_duration)

# Apply the function to create 'From' and 'To' columns
train_set[['From', 'To']] = train_set['Sleep Duration'].apply(lambda x: pd.Series(extract_range(x)))
test_set[['From', 'To']] = test_set['Sleep Duration'].apply(lambda x: pd.Series(extract_range(x)))

# Handle missing values
train_set['From'] = train_set['From'].fillna(train_set['From'].mean())
train_set['To'] = train_set['To'].fillna(train_set['To'].mean())

test_set['From'] = test_set['From'].fillna(test_set['From'].mean())
test_set['To'] = test_set['To'].fillna(test_set['To'].mean())

# Drop the original column if not needed
train_set.drop(columns=['Sleep Duration'], inplace=True)
test_set.drop(columns=['Sleep Duration'], inplace=True)

In [99]:
print("Train set shape:", train_set.shape)
print("Test set shape:", test_set.shape)

Train set shape: (140700, 22)
Test set shape: (93800, 22)


In [100]:
# Define mapping for valid dietary habits
dietary_mapping = {
    'Moderate': 'Moderate',
    'Unhealthy': 'Unhealthy',
    'Healthy': 'Healthy',
    'More Healthy': 'Healthy',
    'Less Healthy': 'Unhealthy',
    'Less than Healthy': 'Unhealthy',
    'No Healthy': 'Unhealthy',
    '3': 'Moderate',
    '1.0': 'Unhealthy',
    '2': 'Moderate',
    '5 Healthy': 'Healthy',
    '5 Unhealthy': 'Unhealthy',

    # Ambiguous or irrelevant entries
    'Yes': None, 'No': None, 'No Healthy': None, 'Class 12': None, 'Indoor': None,
    'Male': None, 'Vegas': None, 'M.Tech': None, 'Electrician': None,
    'Hormonal': None, 'Mihir': None, 'Gender': None, 'BSc': None,
    'Pratham': None, 'Prachi': None, 'Resistant': None, 'Mealy': None,

    'nan': None, 'Academic': None, 'Educational': None, 'Soham': None,
    'Naina': None, 'Kolkata': None, 'Raghav': None, 'Vivaan': None,  'MCA': None,
}

# Apply the mapping to the Dietary Habits column
train_set['Dietary Habits'] = train_set['Dietary Habits'].map(dietary_mapping)
train_set['Dietary Habits'] = train_set['Dietary Habits'].fillna('X')

test_set['Dietary Habits'] = test_set['Dietary Habits'].map(dietary_mapping)
test_set['Dietary Habits'] = test_set['Dietary Habits'].fillna('X')

In [101]:
print("Train set shape:", train_set.shape)
print("Test set shape:", test_set.shape)

Train set shape: (140700, 22)
Test set shape: (93800, 22)


In [102]:
# Check for columns with object data type in train_set and test_set
train_object_columns = train_set.select_dtypes(include=['object']).columns
test_object_columns = test_set.select_dtypes(include=['object']).columns

print("Object columns in train_set:", train_object_columns.tolist())
print("Object columns in test_set:", test_object_columns.tolist())

# Check for columns with missing values in train_set and test_set
train_missing = train_set.isna().sum()
test_missing = test_set.isna().sum()

print("\nColumns with missing values in train_set:")
print(train_missing[train_missing > 0])

print("\nColumns with missing values in test_set:")
print(test_missing[test_missing > 0])

Object columns in train_set: ['City', 'Profession', 'Dietary Habits', 'Degree']
Object columns in test_set: ['City', 'Profession', 'Dietary Habits', 'Degree']

Columns with missing values in train_set:
Series([], dtype: int64)

Columns with missing values in test_set:
Series([], dtype: int64)


In [103]:
X = train_set.copy()
y = target_variable

# Initial split for training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

# Print the count of True and False in both train and validation sets
print("Training set class distribution:\n", y_train.value_counts())
print("Validation set class distribution:\n", y_val.value_counts())

Training set class distribution:
 Depression
0    103620
1     23010
Name: count, dtype: int64
Validation set class distribution:
 Depression
0    11513
1     2557
Name: count, dtype: int64


In [104]:
categorical_columns = ['City', 'Profession', 'Dietary Habits', 'Degree']
for col in categorical_columns:
    X_train[col] = X_train[col].astype(str)
    X_val[col] = X_val[col].astype(str)
    test_set[col] = test_set[col].astype(str)

In [112]:
# Define CatBoost parameters
params1 = {
    'learning_rate': 0.14053992185033634,
    'depth': 2,
    'iterations': 1000,
    'l2_leaf_reg': 3.54988240208208,
    'subsample': 0.5,
    'colsample_bylevel': 1.0,
    'loss_function': 'Logloss',
    'eval_metric': 'Accuracy',
    'verbose': 100
}

# Initialize CatBoostClassifier and specify categorical features
cat_model1 = CatBoostClassifier(cat_features=categorical_columns, **params1)

# Train the model
cat_model1.fit(X_train, y_train, eval_set=(X_train, y_train), early_stopping_rounds=50)

# Predictions on train set
y_pred_cat_train = cat_model1.predict(X_train)
print("Classification Report - Train:")
print(classification_report(y_train, y_pred_cat_train))

# Predictions on validation set
y_pred_cat_val = cat_model1.predict(X_val)
print("Classification Report - Validation:")
print(classification_report(y_val, y_pred_cat_val))

# Predictions on test set
y_pred_cat_test = cat_model1.predict(test_set)


0:	learn: 0.9060254	test: 0.9060254	best: 0.9060254 (0)	total: 23.2ms	remaining: 23.2s
100:	learn: 0.9380794	test: 0.9380952	best: 0.9380952 (100)	total: 1.72s	remaining: 15.4s
200:	learn: 0.9393982	test: 0.9394693	best: 0.9395720 (197)	total: 3.37s	remaining: 13.4s
300:	learn: 0.9400853	test: 0.9400853	best: 0.9400853 (299)	total: 5.04s	remaining: 11.7s
400:	learn: 0.9402116	test: 0.9403301	best: 0.9403301 (400)	total: 6.76s	remaining: 10.1s
500:	learn: 0.9403380	test: 0.9404564	best: 0.9404564 (491)	total: 8.52s	remaining: 8.49s
600:	learn: 0.9406855	test: 0.9407486	best: 0.9407723 (597)	total: 10.3s	remaining: 6.85s
700:	learn: 0.9408197	test: 0.9408355	best: 0.9408908 (657)	total: 12.1s	remaining: 5.18s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9408907842
bestIteration = 657

Shrink model to first 658 iterations.
Classification Report - Train:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96    103620
     

In [ ]:
from google.colab import files
# Prepare the output DataFrame
output = pd.DataFrame({'id': test_id.values, 'loan_status': y_pred_prob_XGB1_test})

# Remove any duplicate rows by 'PassengerId'
output.drop_duplicates(subset='id', keep='first', inplace=True)

# Save predictions to a CSV file
output.to_csv('predictions.csv', index=False)
files.download('predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>